In [1]:
import numpy as np
import glob
import re
import functools
import operator
from collections import Counter
# Step 0: Scan all text files
paths = glob.glob('data/*.txt')

In [2]:
def getIndex(vocab, word):
    index = vocab.get(word, len(vocab))
    if index == len(vocab):
        vocab[word] = index
    return index

In [3]:
contents = []
dictionary = dict()
df = dict()
for path in paths:
    # Read file
    f = open(path, encoding="utf8")
    str = f.read()
    words = re.findall(r"[\w']+", str)
    words = [getIndex(dictionary, word.lower()) for word in words]
    contents.append(words)

In [4]:
term_doc = np.zeros((1, len(dictionary) * len(paths)))
lf = lambda i, x: list(map(lambda num: num * len(paths) + i, x))
index = list(map(lambda x: lf(x[0], x[1]), list(enumerate(contents))))
index = functools.reduce(operator.iconcat, index, [])
frq = Counter(index)
np.put(term_doc, list(frq.keys()), list(frq.values()))
term_doc = np.reshape(term_doc, (len(dictionary), len(paths)))

In [5]:
query = "Xiaomi ra Mi 11 Lite"
query = query.lower()

In [6]:
idf = np.array([np.count_nonzero(term_doc, axis=1)]).transpose()
term_doc = (np.log(term_doc, where=term_doc > 0) + 1 - np.where(term_doc == 0, 1, 0)) * (1 + np.log(len(paths)) - np.log(idf))

In [7]:
norm = np.sqrt(np.sum(term_doc * term_doc, axis=0))
term_doc = term_doc / norm

In [8]:
term_doc

array([[0.04562626, 0.0225024 , 0.        , ..., 0.03143849, 0.        ,
        0.02675491],
       [0.02628997, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.04180721, 0.06458072, 0.        , ..., 0.05639324, 0.        ,
        0.05139823],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.07376777],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.07376777],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.07376777]])

In [9]:
query = re.findall(r"[\w]+", query)
query = Counter(query)
query

Counter({'xiaomi': 1, 'ra': 1, 'mi': 1, '11': 1, 'lite': 1})

In [10]:
vec = np.zeros((len(dictionary), 1))
for word, freq in query.items():
    if word in dictionary:
        vec[dictionary[word], 0] = 1 + np.log(freq)
vec = vec * (1 + np.log(len(paths)) - np.log(idf))

In [11]:
res = np.sum(term_doc * vec, axis=0) / np.sqrt(np.sum(vec * vec))
res

array([0.01047263, 0.        , 0.        , 0.00837919, 0.01152414,
       0.00399946, 0.14837632, 0.01043734, 0.        , 0.00422394,
       0.        , 0.        , 0.00508865, 0.02175583, 0.00851142,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.29040595, 0.        , 0.00572756, 0.00305132,
       0.        , 0.        , 0.01265665, 0.03992644, 0.02104688,
       0.        , 0.01321781, 0.0116633 , 0.0098286 , 0.        ,
       0.01710296, 0.02539677, 0.01236368, 0.00791383, 0.02188656,
       0.01179279, 0.00643239, 0.00945711, 0.        , 0.00660092,
       0.        , 0.0203555 ])

In [22]:
l = list(range(len(paths)))
l.sort(key=lambda x : res[x], reverse=True)
l
for x in l:
    print(paths[x])

data/data10.txt
data/data9.txt
data/data11.txt
data/data37.txt
data/data45.txt
data/data31.txt
data/data46.txt
data/data25.txt
data/data33.txt
data/data39.txt
data/data16.txt
data/data12.txt
data/data22.txt
data/data26.txt
data/data5.txt
data/data42.txt
data/data8.txt
data/data28.txt
data/data24.txt
data/data34.txt
data/data43.txt
data/data20.txt
data/data36.txt
data/data18.txt
data/data40.txt
data/data30.txt
data/data15.txt
data/data23.txt
data/data17.txt
data/data38.txt
data/data6.txt
data/data32.txt
data/data7.txt
data/data41.txt
data/data21.txt
data/data4.txt
data/data44.txt
data/data29.txt
data/data19.txt
data/data13.txt
data/data35.txt
data/data3.txt
data/data27.txt
data/data14.txt
data/data1.txt
data/data47.txt
data/data2.txt
